In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

DATADIR = "SkinData/train"

CATEGORIES = ["benign","malignant"]
IMG_SIZE = 50



In [2]:
training_data = []

def create_training_data():
    for category in CATEGORIES:  
        path = os.path.join(DATADIR,category)  # create path to benign and malignant
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):  # iterate over each image per category
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)  # convert to array
    #             img_array_RGB = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
                new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
                training_data.append([new_array, class_num])
    #             plt.imshow(img_array, cmap="gray")  # graph it
    #             plt.show()  # display!
            except Exception as e:
                pass

create_training_data()


In [3]:
import random
random.shuffle(training_data)

In [4]:
for sample in training_data:
    print(sample[1])

1
1
0
0
0
0
0
0
1
0
1
1
1
0
1
0
1
1
0
0
0
0
0
1
0
1
1
0
0
1
1
0
0
0
1
0
0
0
0
1
1
0
1
1
0
1
1
0
0
0
0
1
0
0
1
0
0
1
0
1
1
1
1
0
1
0
0
1
0
1
0
1
0
1
1
1
0
0
1
1
0
1
0
0
0
1
0
1
0
0
0
1
0
1
1
0
0
0
0
1
0
1
0
1
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
1
1
0
0
0
0
1
0
1
0
1
0
0
0
0
0
1
1
1
1
1
1
0
0
0
1
1
1
0
0
0
1
0
1
1
0
0
0
1
0
0
1
0
0
1
0
0
0
1
0
0
0
0
0
0
1
0
1
0
1
1
1
0
0
0
0
0
0
1
0
1
0
1
0
1
0
1
1
0
0
0
0
1
0
0
1
1
0
0
0
1
1
0
0
1
0
1
0
0
1
0
0
1
0
1
0
0
1
1
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
0
1
0
0
1
0
0
1
1
0
0
0
1
1
1
0
1
1
0
1
1
1
1
0
0
1
1
0
1
1
0
1
1
1
1
0
0
0
1
0
0
1
0
1
1
0
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
1
1
0
1
0
0
0
1
0
0
1
0
0
0
1
1
1
1
0
1
0
1
0
0
0
0
0
1
1
1
1
1
0
0
0
1
1
1
0
0
0
1
0
0
0
1
0
0
1
1
1
1
0
0
0
0
1
1
1
1
1
0
0
1
0
0
1
0
1
1
0
1
0
0
0
1
0
1
0
0
1
0
0
0
1
1
0
1
0
1
0
1
0
0
1
0
0
0
0
1
0
0
0
0
1
1
1
1
0
1
0
0
0
1
1
0
0
0
0
0
1
0
0
1
1
0
0
1
0
0
0
0
1
1
0
1
1
0
0
0
1
0
0
0
1
0
1
0
1
0
0
1
0
0
1
1
1
1
0
1
0
1
0
0
0
0
1
0
0
1
1
1
1
1
1
0
1
0
0
1
1
1
0
1
0
0
0
0
1
1
1
1
1
0
0


In [5]:
print(len(training_data))

2637


In [6]:
X=[]
y=[]

for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

In [7]:
np.save('features.npy',X) #saving
np.save('label.npy',y) #saving
X=np.load('label.npy')#loading
X=np.load('features.npy')#loading

In [8]:
X[1]

array([[[138],
        [123],
        [124],
        ...,
        [144],
        [189],
        [175]],

       [[136],
        [126],
        [132],
        ...,
        [175],
        [167],
        [174]],

       [[133],
        [125],
        [127],
        ...,
        [162],
        [149],
        [191]],

       ...,

       [[177],
        [188],
        [172],
        ...,
        [164],
        [159],
        [173]],

       [[177],
        [175],
        [169],
        ...,
        [159],
        [154],
        [178]],

       [[174],
        [144],
        [185],
        ...,
        [159],
        [167],
        [175]]], dtype=uint8)

In [10]:
X = X/255.0


In [18]:
X.shape[1:]

(50, 50, 1)

In [16]:
NAME = "benign-vs-malignant"

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.4))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

tensorboard = TensorBoard(log_dir="logs\{}".format(NAME))

In [17]:
model.fit(X, y, batch_size=32, epochs=3, validation_split=0.3, callbacks=[tensorboard])

ValueError: A target array with shape (2637, 1) was passed for an output of shape (None, 10) while using as loss `binary_crossentropy`. This loss expects targets to have the same shape as the output.